### Setup

In [ ]:
pip install stocktrends

In [63]:
import yfinance as yf
import numpy as np
from stocktrends import Renko

### Preliminary

Hisse senetleri

In [102]:
tickers = ["ISCTR.IS", "TCELL.IS", "TUPRS.IS", "GARAN.IS", "EREGL.IS"]

ohlcv_data adlı bir dictionary, her bir hisse senedi için 5 dakikalık aralıklarla 1 aylık periyotlar için indirilen OHLCV dataframe verilerini depolar.

dropna() yöntemi, herhangi bir eksik değeri olan satırları kaldırır

In [ ]:
ohlcv_data = {}
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp

## ATR

In [104]:
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

Her bir hisse senedi için ATR hesaplamasını yapıp bu değerleri ohlcv_data dictionary'sinden ilgili hisse senedinin dataframe'ine ekler.

In [105]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]["ATR"] = ATR(ohlcv_data[ticker])

## ADX

In [106]:
def ADX(DF, n=20):
    "function to calculate ADX"
    df = DF.copy()
    df["ATR"] = ATR(DF, n)
    df["upmove"] = df["High"] - df["High"].shift(1)
    df["downmove"] = df["Low"].shift(1) - df["Low"]
    df["+dm"] = np.where((df["upmove"]>df["downmove"]) & (df["upmove"] >0), df["upmove"], 0)
    df["-dm"] = np.where((df["downmove"]>df["upmove"]) & (df["downmove"] >0), df["downmove"], 0)
    df["+di"] = 100 * (df["+dm"]/df["ATR"]).ewm(alpha=1/n, min_periods=n).mean()
    df["-di"] = 100 * (df["-dm"]/df["ATR"]).ewm(alpha=1/n, min_periods=n).mean()
    df["ADX"] = 100* abs((df["+di"] - df["-di"])/(df["+di"] + df["-di"])).ewm(alpha=1/n, min_periods=n).mean()
    return df["ADX"]

Her bir hisse senedi için ADX hesaplamasını yapıp bu değerleri ohlcv_data dictionary'sine ekler.

In [107]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]["ADX"] = ADX(ohlcv_data[ticker],20)

## Bollinger Bands

In [108]:
def Boll_Band(DF, n=14):
    "function to calculate Bollinger Band"
    df = DF.copy()
    df["MB"] = df["Adj Close"].rolling(n).mean()
    df["UB"] = df["MB"] + 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["LB"] = df["MB"] - 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["BB_Width"] = df["UB"] - df["LB"]
    return df[["MB","UB","LB","BB_Width"]]

Bollinger Bantları'nı hesaplar ve bu değerleri ilgili hisse senedinin dataframe'ine ekler.

In [109]:
for ticker in ohlcv_data:
    ohlcv_data[ticker][["MB","UB","LB","BB_Width"]] = Boll_Band(ohlcv_data[ticker])

## MACD

In [110]:
def MACD(DF, a=12 ,b=26, c=9):
    """function to calculate MACD
       typical values a(fast moving average) = 12; 
                      b(slow moving average) =26; 
                      c(signal line ma window) =9"""
    ## exponential moving average  - ewm
    df = DF.copy()
    df["ma_fast"] = df["Adj Close"].ewm(span=a, min_periods=a).mean()
    df["ma_slow"] = df["Adj Close"].ewm(span=b, min_periods=b).mean()
    df["macd"] = df["ma_fast"] - df["ma_slow"]
    df["signal"] = df["macd"].ewm(span=c, min_periods=c).mean()
    return df.loc[:,["macd","signal"]]  

MACD hesaplar ve bu değerleri ilgili hisse senedinin dataframe'ine ekler.

In [111]:
for ticker in ohlcv_data:
    ohlcv_data[ticker][["MACD","SIGNAL"]] = MACD(ohlcv_data[ticker], a=12 ,b=26, c=9)

## RSI

In [112]:
def RSI(DF, n=14):
    "function to calculate RSI"
    df = DF.copy()
    df["change"] = df["Adj Close"] - df["Adj Close"].shift(1)
    df["gain"] = np.where(df["change"]>=0, df["change"], 0)
    df["loss"] = np.where(df["change"]<0, -1*df["change"], 0)
    df["avgGain"] = df["gain"].ewm(alpha=1/n, min_periods=n).mean()
    df["avgLoss"] = df["loss"].ewm(alpha=1/n, min_periods=n).mean()
    df["rs"] = df["avgGain"]/df["avgLoss"]
    df["rsi"] = 100 - (100/ (1 + df["rs"]))
    return df["rsi"]

In [113]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]["RSI"] = RSI(ohlcv_data[ticker])

In [114]:
for ticker in ohlcv_data:
 print(ohlcv_data[ticker])

                                Open       High        Low      Close  \
Datetime                                                                
2023-11-16 09:55:00+03:00  20.139999  20.139999  20.139999  20.139999   
2023-11-16 10:00:00+03:00  20.139999  20.260000  20.139999  20.260000   
2023-11-16 10:05:00+03:00  20.260000  20.400000  20.240000  20.400000   
2023-11-16 10:10:00+03:00  20.379999  20.459999  20.360001  20.400000   
2023-11-16 10:15:00+03:00  20.400000  20.559999  20.379999  20.520000   
...                              ...        ...        ...        ...   
2023-12-15 17:35:00+03:00  24.360001  24.379999  24.280001  24.340000   
2023-12-15 17:40:00+03:00  24.360001  24.400000  24.260000  24.299999   
2023-12-15 17:45:00+03:00  24.320000  24.360001  24.240000  24.260000   
2023-12-15 17:50:00+03:00  24.280001  24.400000  24.219999  24.320000   
2023-12-15 17:55:00+03:00  24.340000  24.360001  24.240000  24.280001   

                           Adj Close   Volume     